In [ ]:
import matplotlib.pyplot as plt
from matplotlib import image
import numpy as np
import pandas as pd
import albumentations as A
import scipy.stats as stats

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from skimage.util import random_noise
from sklearn.model_selection import train_test_split

from keras.backend import set_session
from keras.backend import clear_session
from keras.backend import get_session


from PIL import Image
import glob
import cv2
import os
import shutil
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Visualization functions

In [ ]:
# For plotting accuracy and loss
def plot_metrics_v_epoch(history):
    '''Plots the metrics given the output from model.fit()'''
    plt.figure(figsize=(12,4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper right')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Cross Entropy Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper right')
    
    plt.show()


In [ ]:
def plot_param_metrics(acc_dict, loss_dict):
    plt.figure(figsize=(12,4))
    plt.subplot(1, 2, 1)
    for key in acc_dict.keys():
      plt.plot(acc_dict[key])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(list(acc_dict.keys()), loc='lower right')

    plt.subplot(1, 2, 2)
    for key in loss_dict.keys():
      plt.plot(loss_dict[key])
    plt.title('Model loss')
    plt.ylabel('Cross Entropy Loss')
    plt.xlabel('Epoch')
    plt.legend(list(loss_dict.keys()), loc='upper right')
    
    plt.show()

# Callbacks

In [ ]:
import time
from datetime import datetime

class TimeTakenCallback(tf.keras.callbacks.Callback):
    '''Custom keras callback to implement the timer. Saves the time taken to train 1 epoch in a global variable called training_times'''
    def on_epoch_begin(self, epoch, logs=None):
        self.start = time.time()
    def on_epoch_end(self, epoch, logs=None):
        global training_times
        training_times.append(time.time() - self.start)


time_taken_callback = TimeTakenCallback()
early_stop_callback = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=2, restore_best_weights=True)
callback_list = [time_taken_callback, early_stop_callback]

# Code to change image directory into its desired format

In [ ]:
IMAGE_DIR = '/content/drive/MyDrive/full_data/'
OLD_DIR = '/content/drive/MyDrive/aligned/'

# list of image folders within aligned directory 
image_folders = list(os.listdir(OLD_DIR))[1:]
# gets masterlist of images 
image_names = []
if '.DS_Store' in image_folders:
    image_folders.remove('.DS_Store')
for folder in image_folders:
    if '@' in folder:
        directory = OLD_DIR + str(folder) +"/"
        image_names = image_names + list(os.listdir(directory))[1:]

In [ ]:
# original image directory - labelled as 'aligned' folder from Adience website (https://talhassner.github.io/home/projects/Adience/Adience-data.html#agegender)
# original directory - one folder containing mutiple folders with images 
# required directory - one folder containing all images

def move_images(image_folders, image_names, limit):
    ''' moves images out of smaller folders of old directory into new directory '''
    for i in range(0, limit):
      for filename in image_names[i]:
        face_id = filename.split('.')[1]
        original_image = filename.split('.')[2] + '.jpg'
        
        target = list(data.loc[(data['face_id']==int(face_id)) & (data['original_image']==original_image)]['gender'])[0]
        source = '/content/drive/MyDrive/aligned/' + image_folders[i] + "/" + filename
        destination = IMAGE_DIR + target + '/' + filename
        if target != 'u':
          shutil.copy(source, destination)
    return None

In [ ]:
move_images(image_folders, image_names, len(image_names))

# Importing dataset

In [ ]:
data = pd.read_csv('cleaned_dataset.csv', index_col=0)
data.head()

# 1: Benchmark - Gilnet

In [ ]:
def GilnetModel():
    model = tf.keras.Sequential([
    layers.InputLayer((256,256, 3)),
    layers.Rescaling(1./255.),
    layers.CenterCrop(227, 227),
    # first conv layer
    layers.Conv2D(96, 7, padding='valid'),
    # modified relu to leakyrelu
    layers.LeakyReLU(),
    layers.MaxPooling2D(pool_size=(3, 3), strides=2),
    layers.Lambda(tf.nn.local_response_normalization),  
    # second conv layer
    layers.Conv2D(256, 5, padding='valid'),
    layers.LeakyReLU(),
    layers.MaxPooling2D(pool_size=(3, 3), strides=2),
    layers.Lambda(tf.nn.local_response_normalization),
    # third conv layer
    layers.Conv2D(384, 3, padding='valid'),
    layers.LeakyReLU(),
    layers.MaxPooling2D(pool_size=(3, 3), strides=2),
    layers.Lambda(tf.nn.local_response_normalization),
    layers.Flatten(),
    # first fully connected layer 
    layers.Dense(512),
    layers.LeakyReLU(),
    layers.Dropout(0.5),
    # second fully connected layer 
    layers.Dense(512),
    layers.LeakyReLU(),
    layers.Dropout(0.5),
    # output layer
    layers.Dense(2, activation='softmax')
    ])

    return model

## **Full run**

In [ ]:
# setting parameters 
bs = 32
epochs = 10 
fold = 0
lr = 0.001
training_times = []

idg = ImageDataGenerator()
train_data = image_data.loc[image_data['fold']!=fold].reset_index(drop=True).drop('fold', axis=1)
test_data = image_data.loc[image_data['fold']==fold].reset_index(drop=True).drop('fold', axis=1)
train_data_generator = idg.flow_from_dataframe(train_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))
test_data_generator = idg.flow_from_dataframe(test_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))

model = GilnetModel()
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                    validation_data=test_data_generator, validation_steps=len(test_data)//bs, 
                    epochs=epochs, verbose=1, callbacks = time_taken_callback)

keras.backend.clear_session()

In [ ]:
plot_metrics_v_epoch(history)

# 2: Transfer Learning Model 

## **NASNetMobile**

In [ ]:
def NASNetMobile_tfl():
  global NASNetMobile
  NASNetMobile = tf.keras.applications.NASNetMobile(
      input_shape=(224, 224, 3),
      include_top=False,
      weights="imagenet",
      input_tensor=None,
      pooling=None
  )
  
  inputs = tf.keras.Input(shape=(224, 224, 3))
  x = NASNetMobile(inputs)
  # Convert features of shape `base_model.output_shape[1:]` to vectors
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(128)(x)
  x = layers.LeakyReLU()(x)
  x = layers.Dense(128)(x)
  x = layers.LeakyReLU()(x)
  # A Dense classifier
  outputs = layers.Dense(2, activation='softmax')(x)

  model = tf.keras.Model(inputs, outputs)
  return model

## **Parameter tuning**

In [ ]:
# learning rate tuning on 5000 samples
lr_values = [0.01, 0.001, 0.0001, 0.00001]
lr_accuracies = {}
lr_losses = {}

In [ ]:
# setting parameters 
bs = 32
epochs = 10 
fold = 0
training_times = []

idg = ImageDataGenerator(rescale=1./255)
train_data = image_data.loc[image_data['fold']!=fold].sample(n=4000, random_state=42).reset_index(drop=True).drop('fold', axis=1)
test_data = image_data.loc[image_data['fold']==fold].sample(n=1000, random_state=42).reset_index(drop=True).drop('fold', axis=1)
train_data_generator = idg.flow_from_dataframe(train_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))
test_data_generator = idg.flow_from_dataframe(test_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))

for lr in lr_values:
  model = NASNetMobile_tfl()
  model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                      validation_data=test_data_generator, validation_steps=len(test_data)//bs, 
                      epochs=epochs, verbose=1)
  
  lr_accuracies[lr] = history.history['val_accuracy']
  lr_losses[lr] = history.history['val_loss']
  keras.backend.clear_session()

In [ ]:
plot_param_metrics(lr_accuracies, lr_losses)

In [ ]:
# batch size tuning on 5000 samples
bs_values = [8, 16, 32, 64]
bs_accuracies = {}
bs_losses = {}

In [ ]:
# setting parameters 
lr = 0.00001
epochs = 5
fold = 0
training_times = []

idg = ImageDataGenerator(rescale=1./255)
train_data = image_data.loc[image_data['fold']!=fold].sample(n=4000, random_state=42).reset_index(drop=True).drop('fold', axis=1)
test_data = image_data.loc[image_data['fold']==fold].sample(n=1000, random_state=42).reset_index(drop=True).drop('fold', axis=1)

for bs in bs_values:
  train_data_generator = idg.flow_from_dataframe(train_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))
  test_data_generator = idg.flow_from_dataframe(test_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))
  model = NASNetMobile_tfl()
  model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                      validation_data=test_data_generator, validation_steps=len(test_data)//bs, 
                      epochs=epochs, verbose=1)
  
  bs_accuracies[bs] = history.history['val_accuracy']
  bs_losses[bs] = history.history['val_loss']
  keras.backend.clear_session()

In [ ]:
plot_param_metrics(bs_accuracies, bs_losses)

## **Full run**

In [ ]:
# setting parameters 
bs = 32
epochs = 10 
fold = 0
lr = 1e-5
training_times = []

idg = ImageDataGenerator(rescale=1./255)
train_data = image_data.loc[image_data['fold']!=fold].reset_index(drop=True).drop('fold', axis=1)
test_data = image_data.loc[image_data['fold']==fold].reset_index(drop=True).drop('fold', axis=1)
train_data_generator = idg.flow_from_dataframe(train_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))
test_data_generator = idg.flow_from_dataframe(test_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))

model = NASNetMobile_tfl()
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                    validation_data=test_data_generator, validation_steps=len(test_data)//bs, 
                    epochs=epochs, verbose=1, callbacks = time_taken_callback)

keras.backend.clear_session()

In [ ]:
plot_metrics_v_epoch(history)

## **DenseNet**

In [ ]:
def DenseNet169_tfl():
  global DenseNet169
  DenseNet169 = tf.keras.applications.DenseNet169(
      input_shape=(224, 224, 3),
      include_top=False,
      weights="imagenet",
      input_tensor=None,
      pooling=None
  )
  inputs = tf.keras.Input(shape=(224, 224, 3))
  x = DenseNet169(inputs)
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(128)(x)
  x = layers.LeakyReLU()(x)
  x = layers.Dense(128)(x)
  x = layers.LeakyReLU()(x)
  outputs = layers.Dense(2, activation='softmax')(x)

  model = tf.keras.Model(inputs, outputs)
  return model

In [ ]:
# setting parameters 
bs = 32
epochs = 10 
fold = 0
lr = 1e-5
training_times = []

idg = ImageDataGenerator(rescale=1./255)
train_data = image_data.loc[image_data['fold']!=fold].reset_index(drop=True).drop('fold', axis=1)
test_data = image_data.loc[image_data['fold']==fold].reset_index(drop=True).drop('fold', axis=1)
train_data_generator = idg.flow_from_dataframe(train_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))
test_data_generator = idg.flow_from_dataframe(test_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))

model = DenseNet169_tfl()
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                    validation_data=test_data_generator, validation_steps=len(test_data)//bs, 
                    epochs=epochs, verbose=1, callbacks = time_taken_callback)

In [ ]:
plot_metrics_v_epoch(history)

## **Finetuning**

In [ ]:
bs = 32
epochs = 10
fold = 0
lr = 1e-5
idg = ImageDataGenerator(rescale=1./255)
train_data = image_data.loc[image_data['fold']!=fold].reset_index(drop=True).drop('fold', axis=1)
test_data = image_data.loc[image_data['fold']==fold].reset_index(drop=True).drop('fold', axis=1)
train_data_generator = idg.flow_from_dataframe(train_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))
test_data_generator = idg.flow_from_dataframe(test_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))

# train only fully-connected layers first, with earlystopping
training_times = []
model = DenseNet169_tfl()
DenseNet169.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])
history_1 = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                    validation_data=test_data_generator, validation_steps=len(test_data)//bs, 
                    epochs=epochs, verbose=1, callbacks = callback_list)

training_times_1 = training_times

# then finetune entire model 
DenseNet169.trainable = True # unfreeze layers
lr = 5e-6 # set smaller learning rate
epochs = 5
training_times = []
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])
history_2 = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                    validation_data=test_data_generator, validation_steps=len(test_data)//bs, 
                    epochs=epochs, verbose=1, callbacks = time_taken_callback)
training_times_2 = training_times
keras.backend.clear_session()

In [ ]:
plot_metrics_v_epoch(history_1)

In [ ]:
plot_metrics_v_epoch(history_2)

# 3: Data Augmentation

## Geometric augmentations

In [ ]:
def aug_function(x):
    new_x = x
    # augmentation layers added here
    new_x = layers.RandomFlip(mode="horizontal", seed=42)(new_x)
    new_x = layers.RandomRotation(0.3, seed=42)(new_x)
    new_x = layers.RandomTranslation(0.2,0.2, seed=42)(new_x)
    return new_x

In [ ]:
bs = 32
epochs = 10
fold = 0
lr = 1e-5
idg = ImageDataGenerator(rescale=1./255, preprocessing_function=aug_function)
train_data = image_data.loc[image_data['fold']!=fold].reset_index(drop=True).drop('fold', axis=1)
test_data = image_data.loc[image_data['fold']==fold].reset_index(drop=True).drop('fold', axis=1)
train_data_generator = idg.flow_from_dataframe(train_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))
test_data_generator = idg.flow_from_dataframe(test_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))

training_times = []

model = DenseNet169_tfl()
DenseNet169.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                    validation_data=test_data_generator, validation_steps=len(test_data)//bs, 
                    epochs=epochs, verbose=1, callbacks = time_taken_callback)

keras.backend.clear_session()

In [ ]:
plot_metrics_v_epoch(history)

## **Noise injection**

In [ ]:
def aug_function(x):
    new_x = x

    # adding noise here
    new_x = random_noise(new_x,mode='speckle', mean=0, var=0.05, clip=True)
    new_x = random_noise(new_x, mode='s&p', salt_vs_pepper=0.5, clip=True)
    new_x = layers.GaussianNoise(0.05)(new_x)

    return new_x

In [ ]:
idg = ImageDataGenerator(rescale=1./255, preprocessing_function=aug_function)
train_data = image_data.loc[image_data['fold']!=fold].reset_index(drop=True).drop('fold', axis=1)
test_data = image_data.loc[image_data['fold']==fold].reset_index(drop=True).drop('fold', axis=1)
train_data_generator = idg.flow_from_dataframe(train_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))
test_data_generator = idg.flow_from_dataframe(test_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))

bs = 32
epochs = 10
fold = 0
lr = 1e-5
training_times = []
DenseNet169.trainable = False
model = DenseNet169_tfl()
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                    validation_data=test_data_generator, validation_steps=len(test_data)//bs, 
                    epochs=epochs, verbose=1, callbacks = time_taken_callback)

keras.backend.clear_session()

In [ ]:
plot_metrics_v_epoch(history)

## **Brightness/Contrast/etc.**

In [ ]:
def aug_function(x):
    x = layers.Rescaling(scale=1./255)(x)
    new_x = x.numpy()
    transform = A.Compose([
        A.Blur(p=0.2),
        A.MotionBlur(p=0.2),
        A.HueSaturationValue(p=0.2),
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.2)
    ])
    
    new_x = transform(image=new_x)['image']
    
    return tf.convert_to_tensor(new_x, dtype=tf.float32)

In [ ]:
idg = ImageDataGenerator(preprocessing_function=aug_function)
train_data = image_data.loc[image_data['fold']!=fold].reset_index(drop=True).drop('fold', axis=1)
test_data = image_data.loc[image_data['fold']==fold].reset_index(drop=True).drop('fold', axis=1)
train_data_generator = idg.flow_from_dataframe(train_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))
test_data_generator = idg.flow_from_dataframe(test_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42, target_size=(256, 256))

bs = 32
epochs = 10
fold = 0
lr = 1e-5
training_times = []
DenseNet169.trainable = False
model = DenseNet169_tfl()
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                    validation_data=test_data_generator, validation_steps=len(test_data)//bs, 
                    epochs=epochs, verbose=1, callbacks = time_taken_callback)

keras.backend.clear_session()

In [ ]:
plot_metrics_v_epoch(history)

# 4: Test set oversampling

In [ ]:
def crop_generator(batches, crop_length, position):
    ''' generates the relevant crop of the test image based on position specified '''
    y_start = [16, 0, 31, 0, 31] 
    y_end = [240, 224, 255, 224, 255]
    x_start = [16, 0, 0, 31, 31]
    x_end = [240, 224, 224, 255, 255]
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], x_start[position], x_end[position], y_start[position], y_end[position])
        yield (batch_crops, batch_y)

In [ ]:
def random_crop(img, x_1, x_2, y_1, y_2):
    assert img.shape[2] == 3
    return img[y_1:(y_2), x_1:x_2, :]

In [ ]:
def calculate_accuracy(predicted, actual):
  ''' used calculate validation accuracy using prediction and actual labels '''
  actual = actual[0:len(predicted)-1]
  num_errors = 0
  for p, a in zip(predicted, actual):
    if p != a:
      num_errors += 1
  return (1-num_errors/len(predicted))

In [ ]:
def get_val_accuracy(model):
  predictions = []
  fold = 0
  bs = 32
  global position 

  for position in range(0, 5):
    idg = ImageDataGenerator(rescale=1./255)
    test_data = image_data.loc[image_data['fold']==fold].reset_index(drop=True).drop('fold', axis=1)
    test_data_generator = idg.flow_from_dataframe(test_data, directory = IMAGE_DIR, 
                                                    x_col = "image_file", y_col = "gender",
                                                    class_mode = "categorical", shuffle = False, 
                                                    batch_size=bs, seed=42)
    test_crops = crop_generator(test_data_generator, 224, position)
    new_prediction = model.predict(test_crops, batch_size=bs, verbose=1, steps=len(test_data)//bs)
    gender_dict = {'m':1, 'f':0}
    labels = [*map(gender_dict.get, test_data['gender'])]
    current_predictions = np.argmax(new_prediction, axis=-1)
    predictions.append(new_prediction)

  mean_predictions = np.mean(predictions, axis=0)
  final_predictions = np.argmax(mean_predictions, axis=-1)
  val_accuracy = calculate_accuracy(final_predictions, labels)
  return val_accuracy


In [ ]:
# setting parameters 
bs = 32
epochs = 10
fold = 0
lr = 1e-5
training_times = []

idg = ImageDataGenerator(rescale=1./255)
train_data = image_data.loc[image_data['fold']!=fold].reset_index(drop=True).drop('fold', axis=1)
train_data_generator = idg.flow_from_dataframe(train_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42)
# compiling model
model = DenseNet169_tfl()
DenseNet169.trainable = False # freeze the conv layers
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# train without validation for 5 epochs (due to resource constraints in computing validation accuracy via oversampling)
history = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                    epochs=5, verbose=1)

In [ ]:
accuracy = []
val_accuracy = []
# with test set oversampling
for i in range (0, 5):
  history = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                    epochs=1, verbose=1, callbacks = [time_taken_callback])
  accuracy.append(history.history['accuracy'])
  val_accuracy.append(get_val_accuracy(model))

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')

# 5: Final model

In [ ]:
def aug_function(x):
    x = layers.Rescaling(scale=1./255)(x)
    new_x = x.numpy()
    transform = A.Compose([
        A.Blur(p=0.2),
        A.MotionBlur(p=0.2),
        A.HueSaturationValue(p=0.2),
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.2)
    ])
    
    new_x = transform(image=new_x)['image']
    
    return tf.convert_to_tensor(new_x, dtype=tf.float32)

In [ ]:
# setting parameters 
bs = 32
fold = 0
lr = 1e-5
training_times = []

idg = ImageDataGenerator(preprocessing_function=aug_function)
train_data = image_data.loc[image_data['fold']!=fold].reset_index(drop=True).drop('fold', axis=1)
train_data_generator = idg.flow_from_dataframe(train_data, directory = IMAGE_DIR, 
                                                x_col = "image_file", y_col = "gender",
                                                class_mode = "categorical", shuffle = False, 
                                                batch_size=bs, seed=42)

model = DenseNet169_tfl()
DenseNet169.trainable = False # freeze the conv layers
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# before finetuning
accuracy = []
val_accuracy = []
# 10 epochs
for i in range (0, 10):
  history = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                    epochs=1, verbose=1, callbacks = [time_taken_callback])
  accuracy.append(history.history['accuracy'])
  # get validation accuracy every epoch
  new_val_accuracy = get_val_accuracy(model)
  val_accuracy.append(new_val_accuracy)
  # earlystopping with patience = 1
  if (val_accuracy[-1] <  val_accuracy[-2]):
    break

In [ ]:
# finetuning params and compilation
DenseNet169.trainable = True 
lr = 5e-6
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
ft_accuracy = []
ft_val_accuracy = []
# 5 epochs for finetuning
for i in range (0, 5):
  history = model.fit(train_data_generator, steps_per_epoch=len(train_data)//bs, 
                    epochs=1, verbose=1, callbacks = [time_taken_callback])
  ft_accuracy.append(history.history['accuracy'])
  # every epoch get validation accuracy
  ft_val_accuracy.append(get_val_accuracy(model))

In [ ]:
# transfer learning (before finetuning) plot
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')

In [ ]:
# finetuning plot
plt.plot(ft_accuracy)
plt.plot(ft_val_accuracy)
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')